In [ ]:
import scanpy as sc
import numpy as np

In [ ]:
import os

In [ ]:
os.environ['R_HOME'] = '/home/dbdimitrov/anaconda3/envs/liana4.1/lib/R'

In [ ]:
import rpy2
import rpy2.robjects as robjects

In [ ]:
import anndata2ri
from rpy2.robjects import r
from rpy2.robjects.conversion import localconverter

In [ ]:
from rpy2.robjects.packages import importr, data

In [ ]:
from openproblems.tasks.cell_cell_communication.metrics.odds_ratio import odds_ratio

## FILES

#### Brain

In [ ]:
adata = sc.read("/media/dbdimitrov/SSDDimitrov/Repos/liana2/data/scop/allen_brain.h5ad")

In [ ]:
del adata.layers['logcounts']

In [ ]:
adata.write_h5ad("/home/dbdimitrov/Downloads/allen_brain.h5ad")

In [ ]:
adata.uns["bench"]

In [ ]:
adata

In [ ]:
sc.pp.subsample(adata, n_obs=500)
adata = adata[:, 30000:30500].copy()

In [ ]:
adata

#### TNBC

In [ ]:
adata = sc.read("/media/dbdimitrov/SSDDimitrov/Repos/liana2/data/scop/brca_tnbc.h5ad")

In [ ]:
del adata.layers['logcounts']

In [ ]:
adata

In [ ]:
adata.write_h5ad("/home/dbdimitrov/Downloads/brca_tnbc.h5ad")

### Test Data LOAD, norm!!!

##### BRAIN

In [ ]:
from openproblems.data.allen_brain_atlas import load_mouse_brain_atlas

In [ ]:
adata = load_mouse_brain_atlas(test=True)

In [ ]:
adata

##### TNBC

In [ ]:
from openproblems.data.tnbc_wu2021 import load_tnbc_data
adata = load_tnbc_data(test=True)
adata

Normalize internally in op

In [ ]:
from openproblems.tools.normalize import log_cpm

In [ ]:
adata = log_cpm(adata)
adata.layers['logcounts'] = adata.layers['log_cpm']
del adata.layers['log_cpm']

In [ ]:
adata

#### TEST method + metric!!!!

### RUN LIANA

In [ ]:
import anndata2ri
anndata2ri.activate()
%load_ext rpy2.ipython

In [ ]:
%%R -i adata -o adata
library(tidyverse)
library(liana)

method = "natmi"

adata@assays@data$counts <- as(as.matrix(adata@assays@data$logcounts), "sparseMatrix")
adata@assays@data$logcounts <- as(as.matrix(adata@assays@data$counts), "sparseMatrix")


# Here, we will convert LIANA's Consensus resource to murine symbols
op_resource <- select_resource("Consensus")[[1]]

# Check if the target organism is human
if(adata@metadata$target_organism!=9606){
    # Generate orthologous resource
    op_resource <- generate_homologs(op_resource = op_resource,
                                     target_organism = adata@metadata$target_organism)
}


# Run LIANA
adata@metadata$ccc = liana_wrap(adata,
                                resource = 'custom',
                                external_resource = op_resource,
                                method=method,
                                expr_prop = 0.1,
                                idents_col = "label"
                                ) %>%
    # DEAL WITH THIS AT A PER-METHOD BASIS (in R again) ^ code above is shared for all methods
    mutate(score := .data[[liana:::.score_specs()[[method]]@method_score]]) %>%
    arrange(desc(score))

In [ ]:
check_method(adata)

In [ ]:
# Merge liana's results with the truth
gt = adata.uns["bench"].merge(adata.uns["ccc"], how='right')
adata.uns["truth"] = gt[gt['response'].notna()]

In [ ]:
adata.uns["truth"]

In [ ]:
odds_ratio(adata, top_n=1)

In [ ]:
odds_ratio

In [ ]:
import inspect
lines = inspect.getsource(odds_ratio)
print(lines)


In [ ]:
"ccc" in adata.uns

In [ ]:
all( adata.uns["ccc"]

In [ ]:
set(adata.uns["ccc"])

In [ ]:
"response" in adata.uns["bench"]

### Create Sample dataset

In [ ]:
from openproblems.data.sample import load_sample_data

In [ ]:
adata = load_sample_data()

Modify sample data to required input

In [ ]:
# keep only the top 10 most variable
sc.pp.highly_variable_genes(adata, n_top_genes=10)
adata = adata[:,adata.var['highly_variable']]
# assign names to known interactions
adata.var.index = ['LGALS9', 'PTPRC',
                   'LRP1', 'CD47',
                   'CD44', 'COL1A1',
                   'ADAM10', 'SIRPA',
                  'COL4A1', 'THBS2']
# transfer label
adata.obs["label"] = adata.obs.cell_name

# assign to human prior knowledge
adata.uns["target_organism"] = 9606

In [ ]:
adata = log_cpm(adata)
adata.layers['logcounts'] = adata.layers['log_cpm']
del adata.layers['log_cpm']

In [ ]:
adata.uns['ccc']

In [ ]:
method_sample = pd.DataFrame(rng.integers(0, 100, size=(100, 4)), columns=list('ABCD'))

In [ ]:
import pandas as pd
import random

In [ ]:
?random.sample

In [ ]:
df

In [ ]:
df['NEW'] = pd.Series(pd.util.testing.rands(3), index=df.index)

In [ ]:
df

In [ ]:
num_rows = 10

data = np.array([id_generator() for i in range(2*num_rows)]).reshape(-1,2)
pd.DataFrame(data)


Test tests

In [ ]:
from openproblems.tasks.cell_cell_communication.api import check_dataset, sample_dataset, sample_method, check_method
adata = sample_dataset()

In [ ]:
check_dataset(adata)

In [ ]:
adata = sample_method(adata)

In [ ]:
adata.uns["ccc"]

In [ ]:
check_method(adata)

Test methods

In [ ]:
from openproblems.data.tnbc_wu2021 import load_tnbc_data
adata = load_tnbc_data(test=False)
adata

In [ ]:
adata.write_h5ad("/home/dbdimitrov/Downloads/test.h5ad")

In [ ]:
import scanpy as sc
import numpy as np
import pandas as pd

In [ ]:
adata = sc.read_h5ad("/home/dbdimitrov/Downloads/test.h5ad")

In [ ]:
# adata.X = adata.layers["counts"]

In [ ]:
adata = adata[:, 0:10000]

In [1]:
from openproblems.tasks.cell_cell_communication.methods import liana, natmi, sca, logfc, cellphonedb, connectome

In [2]:
from openproblems.tasks.cell_cell_communication.api import check_dataset, sample_dataset, sample_method, check_method
adata = sample_dataset()

/home/dbdimitrov/Repos/openproblems/openproblems/data/sample.py:23: FutureWarning: X.dtype being converted to np.float32 from int64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  adata = anndata.AnnData(rna_data, obs=cells, var=genes)
/home/dbdimitrov/Repos/openproblems/openproblems/tasks/cell_cell_communication/api.py:60: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs["label"] = adata.obs.cell_name
/home/dbdimitrov/anaconda3/lib/python3.8/contextlib.py:120: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  next(self.gen)


In [3]:
adata = liana(adata, test=True)

/home/dbdimitrov/.local/lib/python3.8/site-packages/scanpy/preprocessing/_normalization.py:182: UserWarning: Some cells have zero counts
  warn(UserWarning('Some cells have zero counts'))
/home/dbdimitrov/anaconda3/lib/python3.8/site-packages/anndata2ri/py2r.py:37: NotConvertedWarning: Conversion 'py2rpy' not defined for objects of type '<class 'NoneType'>'
  warn(str(e), NotConvertedWarning)

    exprs

    combine

    intersect, setdiff, union

    intersect

    collapse, desc, intersect, setdiff, slice, union

    count

    first, intersect, rename, setdiff, setequal, union

    combine, intersect, setdiff, union

    filter, lag

    intersect, setdiff, setequal, union



    an issue that caused a segfault when used with rpy2:
    https://github.com/rstudio/reticulate/pull/1188
    Make sure that you use a version of that package that includes
    the fix.
    

In [4]:
adata.uns["ccc"]

,source,target,ligand,receptor,aggregate_rank,mean_rank,natmi.edge_specificity,natmi.rank,connectome.weight_sc,connectome.rank,logfc.logfc_comb,logfc.rank,sca.LRscore,sca.rank,cellphonedb.pvalue,cellphonedb.rank,score
1,293T,mixed,LGALS9,PTPRC,0.000211,8.6,0.168237,9.0,0.324677,14.0,1.040321,12.0,0.611545,7.0,0.0,1.0,0.000211
2,293T,mixed,ADAM10,CD44,0.001600,10.4,0.159087,12.0,0.378205,6.0,0.819801,21.0,0.605995,12.0,0.0,1.0,0.001600
3,A549,mixed,CD47,SIRPA,0.001900,6.6,0.177890,5.0,0.336299,10.0,1.054802,8.0,0.608364,9.0,0.0,1.0,0.001900
4,A549,mixed,LGALS9,PTPRC,0.002522,9.2,0.155647,14.0,0.301937,17.0,1.365689,1.0,0.602266,13.0,0.0,1.0,0.002522
5,mixed,A549,COL4A1,CD47,0.002759,5.4,0.182090,4.0,0.350557,7.0,1.079703,6.0,0.608364,9.0,0.0,1.0,0.002759
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,3T3,3T3,COL4A1,CD47,1.000000,74.2,0.060116,75.0,-0.055297,83.0,-0.517685,88.0,0.471612,81.0,0.5,44.0,1.000000
76,A549,3T3,COL4A1,CD47,1.000000,76.8,0.058248,76.0,-0.063019,87.0,-0.551847,90.0,0.467682,87.0,0.5,44.0,1.000000
77,3T3,mixed,COL4A1,CD44,1.000000,42.4,0.058193,77.0,0.284761,18.0,0.204973,51.0,0.577679,22.0,0.5,44.0,1.000000
67,3T3,3T3,LGALS9,CD44,1.000000,62.4,0.077017,65.0,0.119192,48.0,0.087278,64.0,0.536337,48.0,1.0,87.0,1.000000


In [5]:
check_method(adata)

True

In [6]:
from openproblems.tasks.cell_cell_communication.metrics.odds_ratio import odds_ratio

In [9]:
odds_ratio(adata, top_n=20)

0.8333333333333334

In [ ]:
adata.uns["bench"]

In [ ]:
np.sum(np.random.binomial(1, 0.2, 30))

In [ ]:
adata.uns["ccc"]

In [ ]:
adata

In [ ]:

from random import choices
from random import seed

In [ ]:
pd.DataFrame({'team': ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H'],
                   'points': [18, 22, 19, 14, 14, 11, 20, 28],
                   'assists': [5, 7, 7, 9, 12, 9, 9, 4],
                   'rebounds': [11, 8, 10, 6, 6, 5, 9, 12]})

In [ ]:
choices(list(set(adata.obs.label)), k=20)

In [ ]:
adata.uns["bench"]

In [ ]:
adata.uns["ccc"]

In [ ]:
adata.uns["bench"]

In [ ]:
gt = adata.uns["bench"].merge(adata.uns["ccc"], how='right')
gt = gt[gt['response'].notna()]

In [ ]:
np.sum(gt.response.values)

In [ ]:
gt

In [ ]:
sum(gt.response)

In [ ]:
adata.uns["ccc"]

In [ ]:
adata.uns["bench"]

In [ ]:
?pd.merge(adata.uns["ccc"], how='right')

In [ ]:
adata.uns["bench"].merge(adata.uns["ccc"], how='right', on=['target'])

In [ ]:
adata.uns["ccc"]

In [ ]:
len(gt.response)

In [ ]:
from openproblems.tasks.cell_cell_communication.metrics.odds_ratio import odds_ratio

In [ ]:
odds_ratio(adata, top_n=200)

In [ ]:
( 49/451) / (80/1050)

In [ ]:
import scipy.stats as stats

In [ ]:
# Join benchmark (assumed truth) and ccc results
# Get /w ccc_results and a response [0, 1] column
gt = adata.uns["bench"].merge(adata.uns["ccc"], how="right")
gt = gt[gt["response"].notna()]

In [ ]:
gt

In [ ]:
top_n = 500
# assign the top rank interactions to 1
a = np.zeros(len(gt["score"]))
a[0:top_n] = 1
gt.loc[:, ["top_n"]] = a

# Shape to contingency table
table = np.array(gt.pivot_table(index=["top_n", "response"], aggfunc="size"))

# # if positive or negative class is not in top_n
# if table.shape != (4,):
#     return 1

# Fisher ET
oddsratio, _ = stats.fisher_exact(table.reshape(2, 2))

In [ ]:
oddsratio

In [ ]:
table

In [ ]:
gt

In [ ]:
np.sum(gt.response)

In [ ]:
sum(gt.head(500).response)

In [ ]:
gt